In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from pandas import read_csv
import math
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from keras.layers.core import Dense, Activation, Dropout
import time #helper libraries
from tensorflow.keras import regularizers, optimizers
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
class ForecastingToolkit(object):
    def __init__(self, df = None, model = None):
        """
        Variables that we passed into our functions should now be defined 
        as class attributes, i.e. class variables. 
        """
        # here are a few to get you started 
        
        # store data here
        self.df = df
        self.Y_train_predict = None
        self.Y_test_predict = None
        
        # store your forecasting model here
        self.model = model
        
        # store feature scalers here
        self.scaler_data = None
        self.scaler_dict = None

        # store the training results of your model here
        self.history = None
    
    def load_transform_data(self): 
        # load the energy data
        energy_filepath = '/content/power_usage_2016_to_2020.csv'
        weather_filepath = '/content/weather_2016_2020_daily.csv'
        df_energy = pd.read_csv(energy_filepath, parse_dates=['StartDate'], index_col='StartDate')
        df_energy_day = df_energy.resample('D').sum()

        # # load the weather data
        df_weather = pd.read_csv(weather_filepath, parse_dates=['Date'], index_col='Date')

        # merge the two dataframes
        df = pd.merge(df_energy_day, df_weather, left_index=True, right_index=True)

        # cleanup
        df = df.drop(columns=['day_of_week_x', 'Day'])

        # rename column
        df = df.rename(columns={'Value (kWh)': 'kwh'})

        # truncate the dates (trim early and late dates)
        # to drop data that's only weekly
        self.df = df.loc["2016-06-01": '2020-01-01']

    def scale_data(self):
        scaler_dict = {}
        scaled_data = {}
    
        for col in self.df.columns:
        
          # instantiate the scaler class 
          scaler = MinMaxScaler(feature_range=(0, 1))
          
          # reshape to avoid shape errors
          feat = self.df[col].values.reshape(-1, 1)
          
          # scale data
          col_scaled = scaler.fit_transform(feat)
          
          # dictionary with scaled data for each column
          scaled_data[col] = col_scaled.flatten()
          scaler_dict[col] = scaler
        
        # move scaled data from dict to dataframe
        input_cols = ['kwh', 'Temp_avg', 'Dew_avg', 'Press_avg']
        self.scaled_data = pd.DataFrame.from_dict(scaled_data)[input_cols]
        self.scaler_dict = scaler_dict
    
    def invert_scaling(self, data, output_feat_name):
        self.Y_train = self.scaler_dict[output_feat_name].inverse_transform(self.Y_train)
        self.Y_test = self.scaler_dict[output_feat_name].inverse_transform(self.Y_test)
        self.Y_train_predict = self.scaler_dict[output_feat_name].inverse_transform(self.Y_train_predict)
        self.Y_test_predict = self.scaler_dict[output_feat_name].inverse_transform(self.Y_test_predict)
    
    def create_dataset(self, data, look_back=None, look_ahead=None, predict_only_last=None):
        X_data, Y_data = [], []
        n_samples = len(data)
        window_length = look_back + look_ahead 
        n_sequences = n_samples - window_length + 1
        print(f'Creating {n_sequences} X, Y samples')
        if data.shape[1] > 1:
            y_data = data[:, 0]
        else:
            y_data = data

        for i in range(n_sequences):
            x = data[i : i+look_back]
            y = y_data[i+look_back : i + look_back + look_ahead] 

            if(predict_only_last):
              y = y[-1] 
            X_data.append(x)
            Y_data.append(y)
        
        return np.array(X_data), np.array(Y_data)
    
    def create_train_test_split(self):
        """
        Creates a train test split for sequential data used for time series forecasting. 
        """
        look_back = 28
        look_ahead = 7
        train_size = 0.70
        predict_only_last = False
        df = self.scaled_data
        # calculate the number of training samples 
        n_samples = df.shape[0]
        train_size = int(n_samples * train_size)

        # consecutive samples indexed from zero up to but not including train_size are the training samples 
        train = df.iloc[:train_size].values
        
        # consecutive samples indexed from train_size up to the end are the test samples 
        test = df.iloc[train_size:].values

        # create input and output splits 
        X_train, Y_train = self.create_dataset(train, look_back=look_back, look_ahead=look_ahead, predict_only_last=predict_only_last)
        X_test, Y_test = self.create_dataset(test, look_back=look_back, look_ahead=look_ahead, predict_only_last=predict_only_last)
        
        return X_train, Y_train, X_test, Y_test
      
    def build_model(self):
        # this model architecture is arbitrary - you can experiment with different architectures to see how it affects the score (i.e. gridsearch)

        # set hyperparameter values
        epochs = 25
        batch_size = 32
        dropout_prob = 0.5
        look_ahead = 7

        # input shape is look_back rows by n_feats columns, for each element of the batch
        input_shape = (28, 4)

        # set learning rate and optimizer
        opt = optimizers.Nadam(learning_rate=0.01)

        # Create and train model here
        model = Sequential()

        # single LSTM layer
        model.add(LSTM(256, input_shape=input_shape, activation='tanh', return_sequences=False))

        # add dropout regularization
        model.add(Dropout(dropout_prob))

        # output layer WE'RE DOING REGRESSION, we want to predict continuous data
        model.add(Dense(look_ahead, activation='relu'))

        # compile model
        model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_absolute_error'])
        model.summary()

        self.model = model
    
    def fit_model(self):
        early_stopping = EarlyStopping(monitor='loss', patience=10, min_delta=1.e-6)
        X_train, Y_train, X_test, Y_test = self.create_train_test_split()
        print(X_train.shape, Y_train.shape, X_test.shape, Y_test.shape)
        history = self.model.fit(X_train, 
                            Y_train,
                            epochs=100,
                            batch_size=32,
                            verbose=1,
                            validation_data=(X_test, Y_test),
                            callbacks = [early_stopping])
        self.history = history
    
    def predict(self):
        # make predictions on train and test inputs 
        X_train = self.X_train
        X_test = self.X_test
        Y_train_predict = self.model.predict(X_train)
        Y_test_predict = self.model.predict(X_test)
    
    def plot_model_loss_metrics(self):
        """
        Use the model history callback to plot the train and test losses vs epochs as well as metrics vs. epochs 
        """
        history = self.history
        # plot training and test loss scores 
        test_loss = history.history["val_loss"]
        train_loss = history.history["loss"]
        
        test_mae = history.history["val_mean_absolute_error"]
        train_mae = history.history["mean_absolute_error"]
        
        n_epochs = len(test_loss) + 1
        epochs = np.arange(1,  n_epochs)
        y_ticks = np.arange(0, 1, 11)

        plt.figure(figsize=(20,5))
        plt.title("Loss vs. Number of Epochs")
        plt.plot(epochs[1:], test_loss[1:], label = "Test Loss")
        plt.plot(epochs[1:], train_loss[1:], label = "Train Loss")
        plt.xlim(1,20)
        plt.xticks(epochs[1:])
        plt.grid()
        plt.legend()
        plt.show() 
        
        plt.figure(figsize=(20,5))
        plt.title("mean_absolute_error vs. Number of Epochs")
        plt.plot(epochs[1:], test_mae[1:], label = "Test MAE")
        plt.plot(epochs[1:], train_mae[1:], label = "Train MAE")
        plt.xlim(1,20)
        plt.xticks(epochs[1:])
        plt.grid()
        plt.legend()
        plt.show() 
    

----

In [ ]:
# once you've completed your class, you'll be able to perform a many operations with just a few lines of code!
tstk = ForecastingToolkit()
tstk.load_transform_data()
tstk.scale_data()
tstk.build_model()
tstk.fit_model()
tstk.plot_model_loss_metrics()

Use LSTM to predict future 7 days minimum daily temperatures over 10 years (1981-1990) in the city Melbourne, Australia.

In [ ]:
look_ahead = 7

In [ ]:
def load_transform_data(): 
    # load the energy data
    filepath = 'https://raw.githubusercontent.com/jbrownlee/Datasets/master/daily-min-temperatures.csv'
    df = pd.read_csv(filepath, parse_dates=['Date'], index_col='Date')
    return df
df = load_transform_data()

In [ ]:
def scale_data(df):
    scaler_dict = {}
    scaled_data = {}

    for col in df.columns:
    
      # instantiate the scaler class 
      scaler = MinMaxScaler(feature_range=(0, 1))
      
      # reshape to avoid shape errors
      feat = df[col].values.reshape(-1, 1)
      
      # scale data
      col_scaled = scaler.fit_transform(feat)
      
      # dictionary with scaled data for each column
      scaled_data[col] = col_scaled.flatten()
    
    # move scaled data from dict to dataframe
    return pd.DataFrame.from_dict(scaled_data)
df = scale_data(df)

In [ ]:
def create_dataset(data, look_back=None, look_ahead=None, predict_only_last=None):
    X_data, Y_data = [], []
    n_samples = len(data)
    window_length = look_back + look_ahead 
    n_sequences = n_samples - window_length + 1
    print(f'Creating {n_sequences} X, Y samples')
    '''
    if data.shape[1] > 1:
        y_data = data[:, 0]
    else:
        y_data = data
    '''
    y_data = data[:, 0]
    for i in range(n_sequences):
        x = data[i : i+look_back]
        y = y_data[i+look_back : i + look_back + look_ahead] 

        if(predict_only_last):
          y = y[-1] 
        X_data.append(x)
        Y_data.append(y)
    
    return np.array(X_data), np.array(Y_data)

In [ ]:
def create_train_test(df):
    """
    Creates a train test split for sequential data used for time series forecasting. 
    """
    look_back = 30
    train_size = 0.70
    predict_only_last = False
    # calculate the number of training samples 
    n_samples = df.shape[0]
    train_size = int(n_samples * train_size)

    # consecutive samples indexed from zero up to but not including train_size are the training samples 
    train = df.iloc[:train_size].values
    
    # consecutive samples indexed from train_size up to the end are the test samples 
    test = df.iloc[train_size:].values
    X_train, Y_train = create_dataset(train, look_back=look_back, look_ahead=look_ahead, predict_only_last=predict_only_last)
    X_test, Y_test = create_dataset(test, look_back=look_back, look_ahead=look_ahead, predict_only_last=predict_only_last)
        
    return X_train, Y_train, X_test, Y_test

In [ ]:
X_train, Y_train, X_test, Y_test = create_train_test(df)

In [ ]:
Y_train.shape

In [ ]:
def build_model(epochs = 25, batch_size = 32, dropout_prob = 0.5, look_ahead = 7):
    # this model architecture is arbitrary - you can experiment with different architectures to see how it affects the score (i.e. gridsearch)

    # set hyperparameter values
    epochs = epochs
    batch_size = batch_size
    dropout_prob = dropout_prob
    look_ahead = look_ahead

    # input shape is look_back rows by n_feats columns, for each element of the batch
    input_shape = (30, 1)

    # set learning rate and optimizer
    opt = optimizers.Nadam(learning_rate=0.01)

    # Create and train model here
    model = Sequential()

    # single LSTM layer
    model.add(LSTM(256, input_shape=input_shape, activation='tanh', return_sequences=False))

    # add dropout regularization
    model.add(Dropout(dropout_prob))

    # output layer WE'RE DOING REGRESSION, we want to predict continuous data
    model.add(Dense(look_ahead, activation='relu'))

    # compile model
    model.compile(loss='mean_squared_error', optimizer=opt, metrics=['mean_absolute_error'])
    model.summary()
    return model 

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
model = KerasClassifier(build_fn = build_model)

In [ ]:
%%time
# fit model

# Create and run Grid Search
# build out our hyperparameter dictionary 
early_stopping = EarlyStopping(monitor='loss', patience=10, min_delta=1.e-6)
hyper_parameters = { 
    "epochs": [10, 50, 100],
    "dropout_prob": np.linspace(0.0, 0.6, num=3),
    "callbacks": [early_stopping],
    "validation_data": [(X_test, Y_test)],
    "batch_size": [32]
    } 
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=model, 
                    param_grid=hyper_parameters, 
                    n_jobs=-3, 
                    verbose=1, 
                    cv=2)

grid_result = grid.fit(X_train, Y_train)

In [ ]:
#dir(grid_result)
grid_result.best_params_

In [ ]:
best_model = grid_result.best_estimator_.build_fn(epochs = 10, batch_size = 32, dropout_prob = 0, look_ahead = 7)
history = best_model.fit(X_train, Y_train, 
                    epochs=10, 
                    batch_size=32, 
                    verbose=1,
                    validation_data=(X_test, Y_test),
                    callbacks=[early_stopping])

In [ ]:
history.history

In [ ]:
def plot_model_loss_metrics(history):
    """
    Use the model history callback to plot the train and test losses vs epochs as well as metrics vs. epochs 
    """
    history = history
    # plot training and test loss scores 
    test_loss = history.history["loss"]
    train_loss = history.history["loss"]
    
    test_mae = history.history["val_mean_absolute_error"]
    train_mae = history.history["mean_absolute_error"]
    
    n_epochs = len(test_loss) + 1
    epochs = np.arange(1,  n_epochs)
    y_ticks = np.arange(0, 1, 11)

    plt.figure(figsize=(20,5))
    plt.title("Loss vs. Number of Epochs")
    plt.plot(epochs[1:], test_loss[1:], label = "Test Loss")
    plt.plot(epochs[1:], train_loss[1:], label = "Train Loss")
    plt.xlim(1,20)
    plt.xticks(epochs[1:])
    plt.grid()
    plt.legend()
    plt.show() 
    
    plt.figure(figsize=(20,5))
    plt.title("mean_absolute_error vs. Number of Epochs")
    plt.plot(epochs[1:], test_mae[1:], label = "Test MAE")
    plt.plot(epochs[1:], train_mae[1:], label = "Train MAE")
    plt.xlim(1,20)
    plt.xticks(epochs[1:])
    plt.grid()
    plt.legend()
    plt.show() 

In [ ]:
plot_model_loss_metrics(history)

In [ ]:
def predict(X_train, X_test):
    # make predictions on train and test inputs 
    Y_train_predict = best_model.predict(X_train)
    Y_test_predict = best_model.predict(X_test)
    
    return Y_train_predict, Y_test_predict

Y_train_predict, Y_test_predict = predict(X_train, X_test)

In [ ]:
## YOUR CODE HERE 
from sklearn.preprocessing import MinMaxScaler
def invert_scaling(data):
    scaler = MinMaxScaler(feature_range=(0, 1))
    feat = df['Temp'].values.reshape(-1, 1)
    scaler.fit_transform(feat)
    return scaler.inverse_transform(data)
Y_train = invert_scaling(Y_train)
Y_test = invert_scaling(Y_test)
Y_train_predict = invert_scaling(Y_train_predict)
Y_test_predict = invert_scaling(Y_test_predict)
    

In [ ]:
def plot_predictions(Y_train, Y_train_predict, Y_test, Y_test_predict):
    
    plt.figure(figsize=(20,5))
    plt.title("Training Set: True vs Predicted temp")
    plt.grid()
    plt.plot(Y_train_predict[:,0], label = "Predict", c="r")
    plt.plot(Y_train[:,0], label= "True", c="c")
    plt.xlim((0,300))
    plt.legend();
    
    plt.figure(figsize=(20,5))
    plt.title("Test Set: True vs Predicted temp")
    plt.grid()
    plt.plot(Y_test_predict[:,0], label = "Predict", c="r")
    plt.plot(Y_test[:,0], label= "True", c="c")
    plt.xlim((0,300))
    plt.legend();

plot_predictions(Y_train, Y_train_predict, Y_test, Y_test_predict) 

### Compare Model against a Naive Baseline

In [ ]:
# test_scores
print(f'Scores when forecasting with our LSTM model on the test data:')
for day in range(look_ahead):
  test_score_ = math.sqrt(mean_squared_error(Y_test[:,day], Y_test_predict[:,day],))
  print(f'RMSE = {test_score_:.2f} temp for forecasting {day+1} day(s) in the future')
test_score_all = math.sqrt(mean_squared_error(Y_test, Y_test_predict))
print(f'Mean RMSE = {test_score_all:.2f} temp for forecasting all {look_ahead} look_ahead days in the future')

# naive scores i.e. assuming that future kwh consumption will be the same as today for look_ahead days in the future
print(f'\nCompare with naive baseline scores:')
for day in range(1, look_ahead+1):
  y1 = [Y_test[i][0] for i in range(0,len(Y_test) - day)]
  y2 = [Y_test[j][0] for j in range(day,len(Y_test))]
  naive_score = math.sqrt(mean_squared_error(y1,y2))
  print(f'RMSE = {naive_score:.2f} temp for forecasting {day} day(s) in the future') 